# Questions to the data
1. Does the number of hauses in all houses groups remain stable or there were a construction side finished in the given time frame?
2. What was the weather in the region?
3. What was the price for gas/oil/elictricity?

In [190]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Tuple, Any
import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import warnings
from data.starting_kit.utils import create_submission
from sklego.preprocessing import RepeatingBasisFunction
warnings.filterwarnings("ignore")

In [191]:
data_original = pd.read_csv('../data/public_data/train.csv')
final_index = data_original['pseudo_id']

In [192]:

data_filtered = pd.read_csv('../data/train_filtered.csv')

In [193]:
# drop index for feature preparation
data_ = data_filtered.drop(columns='pseudo_id')
# convert dates to pandas datetime
data_.columns = [datetime.strptime(c, "%Y-%m-%d %H:%M:%S") for c in data_.columns]
data_.head()

,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,...,2019-08-28 19:00:00,2019-08-28 19:30:00,2019-08-28 20:00:00,2019-08-28 20:30:00,2019-08-28 21:00:00,2019-08-28 21:30:00,2019-08-28 22:00:00,2019-08-28 22:30:00,2019-08-28 23:00:00,2019-08-28 23:30:00
0,45.022999,39.985001,36.569500,34.748001,35.972000,38.438999,36.591000,36.315498,32.660500,31.564501,...,24.2880,23.994,26.1995,25.0270,23.0665,26.093,23.4295,25.4715,26.246,22.6020
1,2.931000,1.641000,2.260000,2.273000,2.651000,3.137000,2.532000,3.142000,2.528000,2.637000,...,2.5700,1.446,1.5230,1.5630,2.5880,2.190,1.4860,2.5270,2.288,1.7940
2,11.014000,12.652500,10.824001,13.748500,12.383000,12.342000,13.413000,11.484000,11.510500,11.480751,...,6.3565,5.766,5.4955,5.0885,6.8140,7.492,5.7705,6.8240,6.072,6.7205
3,55.813000,49.040001,49.095001,41.132999,45.660000,48.477001,50.539001,45.737000,42.680000,42.782501,...,32.6460,30.439,30.2470,31.2660,34.3390,33.076,33.1080,33.7260,30.009,34.8400
4,26.925001,28.118000,25.600000,28.091000,26.530001,23.858002,26.556000,27.714001,23.174002,23.545000,...,13.3980,13.280,13.7340,13.6060,14.7000,16.290,15.1240,15.3650,14.360,13.9350


# GROUP BY DAY

In [194]:
# Aggregate energy use values per day
data_ = data_.T.groupby(data_.T.index.date).sum()
data_.columns = [c+1 for c in range(len(data_.columns))]

In [196]:
# check wether datetime in ascending order , it is important for time series
print(data_.index.is_monotonic)

True


In [197]:
# Set dates for development phase
new_date_range = pd.date_range(start="2019-04-01", end="2019-09-04", freq="D")
# Add test dates in the data frame
data_ = data_.reindex(new_date_range)

In [198]:
# saving indexes for next steps
idx_test_date = data_.index[data_[1].isna()]

In [199]:
data_ = data_.fillna(method="ffill")

In [200]:
data_[data_.select_dtypes(np.float64).columns] = data_.select_dtypes(np.float64).astype(np.float32)

In [201]:
#plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':160})
#data_[1].plot()
#plt.title('Energy use forecasts for houshold group' + " 1")
#plt.show()

In [202]:
data_\
    .isna()\
    .sum()\
    .to_frame()\
    .assign(perc = lambda row: 100 * row[0] / data_.shape[0])\
    .rename(columns={0: 'Number of missed data  points', 'perc': '% of missed data points'})

,Number of missed data points,% of missed data points
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
...,...,...
57,0,0.0
58,0,0.0
59,0,0.0
60,0,0.0


# Adding features

In [204]:
df = data_.copy(deep=True)
df["weekday"] = data_.index.weekday
df["dayofyear"] = data_.index.dayofyear
df["month"] = data_.index.month
df["season"] = (data_.index.month % 12 + 3) // 3
df["is_weekend"] = data_.index.weekday.isin([5, 6]).astype(np.int32)

In [205]:
df_month = pd.get_dummies(df["month"], drop_first=True, prefix="month")
df_season = pd.get_dummies(df["season"], drop_first=True, prefix="season")
df_weekday = pd.get_dummies(df["weekday"], drop_first=True, prefix="weekday")
df = pd.concat([df, df_month, df_season, df_weekday], axis=1)
df.drop(columns=['month','season','weekday'], inplace=True)

In [206]:
# rbf = RepeatingBasisFunction(n_periods=12, column="dayofyear", input_range=(1,365), remainder="drop")
# rbf.fit(df)
# X_3 = pd.DataFrame(index=df.index,  data=rbf.transform(df))
# X_3.columns = [str(c)+'_day_month' for c in range(len(X_3.columns))]
# #df = pd.concat([df, X_3], axis=1)

In [211]:
def create_features(houshold_id,df=df) -> pd.DataFrame:
    df_new = df[[houshold_id, 'is_weekend',
                 #'month_2', 'month_3', 'month_4',
                 'month_5', 'month_6', 'month_7','month_8','month_9',
                 #'month_10','month_11','month_12',
                 #'season_2',
                 'season_3','season_4','weekday_1', 'weekday_2', 'weekday_3','weekday_4','weekday_5','weekday_6',
                 #'0_day_month','1_day_month','2_day_month','3_day_month','4_day_month','5_day_month','6_day_month','7_day_month','8_day_month','9_day_month','10_day_month','11_day_month'
                 ]]
    df_new['std'] = df_new[houshold_id].rolling(7).std().fillna(method="backfill")
    df_new['mean'] = df_new[houshold_id].rolling(7).mean().fillna(method="backfill")
    df_new['lag_1'] = df[houshold_id].shift(1).fillna(method="backfill")
    df_new['lag_2'] = df[houshold_id].shift(2).fillna(method="backfill")
    df_new['lag_3'] = df_new[houshold_id].shift(3).fillna(method="backfill")
    df_new['lag_4'] = df_new[houshold_id].shift(4).fillna(method="backfill")
    df_new['lag_5'] = df_new[houshold_id].shift(5).fillna(method="backfill")
    df_new['lag_6'] = df_new[houshold_id].shift(6).fillna(method="backfill")
    df_new['lag_7'] = df_new[houshold_id].shift(7).fillna(method="backfill")
    return df_new

In [212]:
create_features(1)

,1,is_weekend,month_5,month_6,month_7,month_8,month_9,season_3,season_4,weekday_1,...,weekday_6,std,mean,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
2019-04-01,3166.856934,0,0,0,0,0,0,0,0,0,...,0,638.963160,3149.839146,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934
2019-04-02,3846.906738,0,0,0,0,0,0,0,0,1,...,0,638.963160,3149.839146,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934
2019-04-03,3935.979980,0,0,0,0,0,0,0,0,0,...,0,638.963160,3149.839146,3846.906738,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934
2019-04-04,3508.844238,0,0,0,0,0,0,0,0,0,...,0,638.963160,3149.839146,3935.979980,3846.906738,3166.856934,3166.856934,3166.856934,3166.856934,3166.856934
2019-04-05,2712.798584,0,0,0,0,0,0,0,0,0,...,0,638.963160,3149.839146,3508.844238,3935.979980,3846.906738,3166.856934,3166.856934,3166.856934,3166.856934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,1021.899475,1,0,0,0,1,0,1,0,0,...,0,148.552525,1138.856986,1021.899475,1021.899475,1021.899475,1279.736450,1255.452026,1349.212524,1868.805542
2019-09-01,1021.899475,1,0,0,0,0,1,0,1,0,...,1,120.091556,1092.097979,1021.899475,1021.899475,1021.899475,1021.899475,1279.736450,1255.452026,1349.212524
2019-09-02,1021.899475,0,0,0,0,0,1,0,1,0,...,0,97.453216,1058.733329,1021.899475,1021.899475,1021.899475,1021.899475,1021.899475,1279.736450,1255.452026
2019-09-03,1021.899475,0,0,0,0,0,1,0,1,1,...,0,0.000000,1021.899475,1021.899475,1021.899475,1021.899475,1021.899475,1021.899475,1021.899475,1279.736450


In [213]:
def get_weeks(idx)-> List[List[pd._libs.tslibs.timestamps.Timestamp]]:
    idx = list(idx)
    weeks_to_predict = []
    week = []
    for i in range(1,len(idx)+1,1):
        #print(idx[i-1])
        if i%7 == 0:
            week.append(idx[i-1])
            weeks_to_predict.append(week)
            #print(week)
            week = []
        elif i%7 > 0:
            week.append(idx[i-1])
    return weeks_to_predict

In [214]:
weeks_test_date = get_weeks(idx_test_date)

In [216]:
def data_split_accumlated(result, weeks, n, houshold) -> Tuple[Any, Any]:
    print(weeks[n])
    result_splitted = result[result.index < weeks[n][0]]
    result_splitted_features = result_splitted.drop([houshold], axis=1)
    result_splitted_target = result_splitted[houshold]
    print("target: ", result_splitted_target)
    result_splitted_to_predict = result[(result.index >= weeks[n][0]) & (result.index <= weeks[n][6])]
    result_splitted_to_predict_features = result_splitted_to_predict.drop([houshold], axis=1)
    result_splitted_to_predict_target = result_splitted_to_predict[houshold]
    result_splitted_to_predict_target = result_splitted_to_predict_target.to_frame()

    return result_splitted_features, result_splitted_target, result_splitted_to_predict_features, result_splitted_to_predict_target

In [217]:
all_predictions = []

start_time = time.time()

for houshold in data_.columns:
    houshold_predictions = []
    print("***** Houshold " + str(houshold) + " dataset created ****** ")
    for week in range(len(weeks_test_date)):
        #print("week "+  str(week) + " splitting started")
        result = create_features(houshold)
        features, target, features_predict, target_predict = data_split_accumlated(result, weeks_test_date, week, houshold)
        model_linear = LinearRegression()
        model_linear.fit(features, target)
        print("trained on " + str(week))
        prediction = model_linear.predict(features_predict)
        houshold_predictions.append(prediction)
        week_timestamps = weeks_test_date[week]
        target_predict['predict'] = prediction
        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict['predict']
        print("=============================================")
        print(prediction)
        #print(df.loc[week_timestamps[0]:week_timestamps[6],houshold])
        print("=============================================")
    all_predictions.append(houshold_predictions)
print("--- %s seconds ---" % (time.time() - start_time))

***** Houshold 1 dataset created ****** 
[Timestamp('2019-04-29 00:00:00'), Timestamp('2019-04-30 00:00:00'), Timestamp('2019-05-01 00:00:00'), Timestamp('2019-05-02 00:00:00'), Timestamp('2019-05-03 00:00:00'), Timestamp('2019-05-04 00:00:00'), Timestamp('2019-05-05 00:00:00')]
target:  2019-04-01    3166.856934
2019-04-02    3846.906738
2019-04-03    3935.979980
2019-04-04    3508.844238
2019-04-05    2712.798584
2019-04-06    2354.390625
2019-04-07    2523.096924
2019-04-08    2758.358398
2019-04-09    3514.557373
2019-04-10    3665.839600
2019-04-11    3367.100586
2019-04-12    2958.354004
2019-04-13    2422.410889
2019-04-14    2350.132080
2019-04-15    3132.182861
2019-04-16    3797.214600
2019-04-17    2981.989502
2019-04-18    2578.500977
2019-04-19    2462.043457
2019-04-20    2543.442383
2019-04-21    2710.635986
2019-04-22    3561.781738
2019-04-23    3753.532227
2019-04-24    3682.431152
2019-04-25    2756.764893
2019-04-26    2432.269531
2019-04-27    2407.176025
2019-04-2

In [218]:
"""all_predictions = []

start_time = time.time()


for houshold in data_.columns:
    houshold_predictions = []
    print("***** Houshold " + str(houshold) + " dataset created ****** ")
    for week in range(len(weeks_test_date)):
        #print("week "+  str(week) + " splitting started")
        result = create_features(houshold)
        features, target, features_predict, target_predict = data_split_accumlated(result, weeks_test_date, week, houshold)

        clf = linear_model.Lasso(alpha=0.1)
        clf.fit(features, target)

        print("trained on " + str(week))

        prediction_lasso = clf.predict(features_predict)
        houshold_predictions.append(prediction_lasso)

        week_timestamps = weeks_test_date[week]

        target_predict['predict'] = prediction_lasso
        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict['predict']
        print("=============================================")
        print(prediction_lasso)
        #print(df.loc[week_timestamps[0]:week_timestamps[6],houshold])
        print("=============================================")
    all_predictions.append(houshold_predictions)
print("--- %s seconds ---" % (time.time() - start_time))"""

'all_predictions = []\n\nstart_time = time.time()\n\n\nfor houshold in data_.columns:\n    houshold_predictions = []\n    print("***** Houshold " + str(houshold) + " dataset created ****** ")\n    for week in range(len(weeks_test_date)):\n        #print("week "+  str(week) + " splitting started")\n        result = create_features(houshold)\n        features, target, features_predict, target_predict = data_split_accumlated(result, weeks_test_date, week, houshold)\n\n        clf = linear_model.Lasso(alpha=0.1)\n        clf.fit(features, target)\n\n        print("trained on " + str(week))\n\n        prediction_lasso = clf.predict(features_predict)\n        houshold_predictions.append(prediction_lasso)\n\n        week_timestamps = weeks_test_date[week]\n\n        target_predict[\'predict\'] = prediction_lasso\n        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict[\'predict\']\n        print("=============================================")\n        print(prediction

In [219]:
weeks_columns = [week for weeks in weeks_test_date for week in weeks]

[Timestamp('2019-04-29 00:00:00'),
 Timestamp('2019-04-30 00:00:00'),
 Timestamp('2019-05-01 00:00:00'),
 Timestamp('2019-05-02 00:00:00'),
 Timestamp('2019-05-03 00:00:00'),
 Timestamp('2019-05-04 00:00:00'),
 Timestamp('2019-05-05 00:00:00'),
 Timestamp('2019-06-13 00:00:00'),
 Timestamp('2019-06-14 00:00:00'),
 Timestamp('2019-06-15 00:00:00'),
 Timestamp('2019-06-16 00:00:00'),
 Timestamp('2019-06-17 00:00:00'),
 Timestamp('2019-06-18 00:00:00'),
 Timestamp('2019-06-19 00:00:00'),
 Timestamp('2019-07-28 00:00:00'),
 Timestamp('2019-07-29 00:00:00'),
 Timestamp('2019-07-30 00:00:00'),
 Timestamp('2019-07-31 00:00:00'),
 Timestamp('2019-08-01 00:00:00'),
 Timestamp('2019-08-02 00:00:00'),
 Timestamp('2019-08-03 00:00:00'),
 Timestamp('2019-08-29 00:00:00'),
 Timestamp('2019-08-30 00:00:00'),
 Timestamp('2019-08-31 00:00:00'),
 Timestamp('2019-09-01 00:00:00'),
 Timestamp('2019-09-02 00:00:00'),
 Timestamp('2019-09-03 00:00:00'),
 Timestamp('2019-09-04 00:00:00')]

In [220]:
def flatten(hous):
    return [week for weeks in hous for week in weeks]

In [221]:
flatened_predictions = []
for hous in all_predictions:
    flatened_predictions.append(flatten(hous))

In [222]:
df_from_list = pd.DataFrame([i for i in flatened_predictions], columns= [weeks_columns],index = final_index)

In [258]:
df_from_list

,pseudo_id,2019-04-29 00:00:00,2019-04-30 00:00:00,2019-05-01 00:00:00,2019-05-02 00:00:00,2019-05-03 00:00:00,2019-05-04 00:00:00,2019-05-05 00:00:00,2019-06-13 00:00:00,2019-06-14 00:00:00,...,2019-08-01 00:00:00,2019-08-02 00:00:00,2019-08-03 00:00:00,2019-08-29 00:00:00,2019-08-30 00:00:00,2019-08-31 00:00:00,2019-09-01 00:00:00,2019-09-02 00:00:00,2019-09-03 00:00:00,2019-09-04 00:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,2832.502432,3134.833312,2407.564743,2279.955375,2168.578617,2034.394632,2119.672013,2113.654448,2104.672339,...,1850.349223,1859.731462,1853.123921,1114.379323,1172.600542,995.456198,833.622375,834.598900,754.008299,748.848364
1,0x1c9d08cd16fce04790ef900695861e786,152.293983,193.302873,170.908193,168.354048,163.389298,151.823103,155.048866,171.870083,172.482780,...,181.171746,183.439506,185.509512,83.296814,85.783757,77.514457,78.623147,75.843357,75.530368,76.885344
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,488.192809,720.852627,340.613454,280.836270,283.952176,314.855847,266.734999,399.567815,379.415866,...,450.117298,429.697022,424.898409,283.297773,317.327054,208.125140,152.514613,149.982347,166.860417,148.932731
3,0x20158d36236a640cf0524dba149459169,3864.434038,4086.336050,2993.141052,2569.061514,2499.642272,2430.347087,2362.013531,2679.946200,2682.859812,...,2086.854406,2084.398739,2103.140262,1510.470186,1578.569368,1397.192032,1267.558468,1271.768344,1118.684126,1087.123356
4,0xc305005dcb1ed6128d816954c5ab9e7e,1951.526223,2192.285119,2291.603062,2049.497225,1914.532510,1823.972002,1933.438184,1181.873577,1280.883862,...,1412.655120,1483.896388,1486.783592,670.876817,644.417271,530.404369,443.601446,433.451819,390.601364,388.092048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,58.747574,58.016621,48.679425,40.580964,37.543120,42.119432,36.936968,33.048017,36.171847,...,20.562497,20.742351,20.971343,16.632708,17.043289,15.229304,9.195301,8.262386,6.748691,8.211728
57,0x16d1816bc5d185c47de080d4c6a64bc9a,120.188327,133.645119,111.266649,109.731485,99.659354,87.120952,68.694801,105.798670,108.680378,...,87.133253,85.775018,86.914259,52.487205,54.882261,50.541506,49.795860,49.660025,47.925070,48.735677
58,0x14f480f24c435af1b8574c1c6bab38a1c,242.123307,264.059537,80.907908,61.371138,46.249275,28.670661,-3.838886,171.996348,176.811264,...,223.848019,225.027278,225.438672,77.427144,85.688791,74.769686,70.960652,67.013481,61.747130,63.823675
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,59.439102,68.287497,46.230467,38.679805,41.510602,31.825989,23.568513,50.007801,51.958749,...,55.433815,57.290765,56.681622,24.453170,25.391323,24.385403,18.580792,16.986403,17.005269,16.969482


In [223]:
df_from_list.reset_index(inplace = True)
df_from_list = df_from_list.round(10)

In [247]:
df_from_list2 = df_from_list.iloc[:,1:60].clip(lower=0)
df_from_list2


,2019-04-29,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-04,2019-05-05,2019-06-13,2019-06-14,2019-06-15,...,2019-08-01,2019-08-02,2019-08-03,2019-08-29,2019-08-30,2019-08-31,2019-09-01,2019-09-02,2019-09-03,2019-09-04
0,2832.502432,3134.833312,2407.564743,2279.955375,2168.578617,2034.394632,2119.672013,2113.654448,2104.672339,2195.735158,...,1850.349223,1859.731462,1853.123921,1114.379323,1172.600542,995.456198,833.622375,834.598900,754.008299,748.848364
1,152.293983,193.302873,170.908193,168.354048,163.389298,151.823103,155.048866,171.870083,172.482780,186.242427,...,181.171746,183.439506,185.509512,83.296814,85.783757,77.514457,78.623147,75.843357,75.530368,76.885344
2,488.192809,720.852627,340.613454,280.836270,283.952176,314.855847,266.734999,399.567815,379.415866,402.397865,...,450.117298,429.697022,424.898409,283.297773,317.327054,208.125140,152.514613,149.982347,166.860417,148.932731
3,3864.434038,4086.336050,2993.141052,2569.061514,2499.642272,2430.347087,2362.013531,2679.946200,2682.859812,2814.749485,...,2086.854406,2084.398739,2103.140262,1510.470186,1578.569368,1397.192032,1267.558468,1271.768344,1118.684126,1087.123356
4,1951.526223,2192.285119,2291.603062,2049.497225,1914.532510,1823.972002,1933.438184,1181.873577,1280.883862,1267.621791,...,1412.655120,1483.896388,1486.783592,670.876817,644.417271,530.404369,443.601446,433.451819,390.601364,388.092048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,58.747574,58.016621,48.679425,40.580964,37.543120,42.119432,36.936968,33.048017,36.171847,39.943975,...,20.562497,20.742351,20.971343,16.632708,17.043289,15.229304,9.195301,8.262386,6.748691,8.211728
57,120.188327,133.645119,111.266649,109.731485,99.659354,87.120952,68.694801,105.798670,108.680378,109.537971,...,87.133253,85.775018,86.914259,52.487205,54.882261,50.541506,49.795860,49.660025,47.925070,48.735677
58,242.123307,264.059537,80.907908,61.371138,46.249275,28.670661,0.000000,171.996348,176.811264,183.507151,...,223.848019,225.027278,225.438672,77.427144,85.688791,74.769686,70.960652,67.013481,61.747130,63.823675
59,59.439102,68.287497,46.230467,38.679805,41.510602,31.825989,23.568513,50.007801,51.958749,53.818387,...,55.433815,57.290765,56.681622,24.453170,25.391323,24.385403,18.580792,16.986403,17.005269,16.969482


In [263]:
idx = 0
df_from_list2.insert(loc=idx, column= 'pseudo_id', value=final_index)

In [264]:
df_from_list2

,pseudo_id,2019-04-29 00:00:00,2019-04-30 00:00:00,2019-05-01 00:00:00,2019-05-02 00:00:00,2019-05-03 00:00:00,2019-05-04 00:00:00,2019-05-05 00:00:00,2019-06-13 00:00:00,2019-06-14 00:00:00,...,2019-08-01 00:00:00,2019-08-02 00:00:00,2019-08-03 00:00:00,2019-08-29 00:00:00,2019-08-30 00:00:00,2019-08-31 00:00:00,2019-09-01 00:00:00,2019-09-02 00:00:00,2019-09-03 00:00:00,2019-09-04 00:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,2832.502432,3134.833312,2407.564743,2279.955375,2168.578617,2034.394632,2119.672013,2113.654448,2104.672339,...,1850.349223,1859.731462,1853.123921,1114.379323,1172.600542,995.456198,833.622375,834.598900,754.008299,748.848364
1,0x1c9d08cd16fce04790ef900695861e786,152.293983,193.302873,170.908193,168.354048,163.389298,151.823103,155.048866,171.870083,172.482780,...,181.171746,183.439506,185.509512,83.296814,85.783757,77.514457,78.623147,75.843357,75.530368,76.885344
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,488.192809,720.852627,340.613454,280.836270,283.952176,314.855847,266.734999,399.567815,379.415866,...,450.117298,429.697022,424.898409,283.297773,317.327054,208.125140,152.514613,149.982347,166.860417,148.932731
3,0x20158d36236a640cf0524dba149459169,3864.434038,4086.336050,2993.141052,2569.061514,2499.642272,2430.347087,2362.013531,2679.946200,2682.859812,...,2086.854406,2084.398739,2103.140262,1510.470186,1578.569368,1397.192032,1267.558468,1271.768344,1118.684126,1087.123356
4,0xc305005dcb1ed6128d816954c5ab9e7e,1951.526223,2192.285119,2291.603062,2049.497225,1914.532510,1823.972002,1933.438184,1181.873577,1280.883862,...,1412.655120,1483.896388,1486.783592,670.876817,644.417271,530.404369,443.601446,433.451819,390.601364,388.092048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,58.747574,58.016621,48.679425,40.580964,37.543120,42.119432,36.936968,33.048017,36.171847,...,20.562497,20.742351,20.971343,16.632708,17.043289,15.229304,9.195301,8.262386,6.748691,8.211728
57,0x16d1816bc5d185c47de080d4c6a64bc9a,120.188327,133.645119,111.266649,109.731485,99.659354,87.120952,68.694801,105.798670,108.680378,...,87.133253,85.775018,86.914259,52.487205,54.882261,50.541506,49.795860,49.660025,47.925070,48.735677
58,0x14f480f24c435af1b8574c1c6bab38a1c,242.123307,264.059537,80.907908,61.371138,46.249275,28.670661,0.000000,171.996348,176.811264,...,223.848019,225.027278,225.438672,77.427144,85.688791,74.769686,70.960652,67.013481,61.747130,63.823675
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,59.439102,68.287497,46.230467,38.679805,41.510602,31.825989,23.568513,50.007801,51.958749,...,55.433815,57.290765,56.681622,24.453170,25.391323,24.385403,18.580792,16.986403,17.005269,16.969482


In [248]:
df_from_list2.to_csv("./sample_submission_DAILY_max_new_features_FILTERED_float32_ROUND_CLIPPED.csv", index = False)

In [249]:
hourly = pd.read_csv("/home/max/PycharmProjects/ai-cup-makcfd/Linear_regression/sample_submission_HOURLY_max_eval_new_features_FILTERED_float32_ROUND.csv")

In [252]:
hourly2 = hourly.iloc[:,1:].clip(lower=0)

In [265]:
idx = 0
hourly2.insert(loc=idx, column= 'pseudo_id', value=final_index)
hourly2

,pseudo_id,2019-04-29 00:00:00,2019-04-29 01:00:00,2019-04-29 02:00:00,2019-04-29 03:00:00,2019-04-29 04:00:00,2019-04-29 05:00:00,2019-04-29 06:00:00,2019-04-29 07:00:00,2019-04-29 08:00:00,...,2019-09-04 14:00:00,2019-09-04 15:00:00,2019-09-04 16:00:00,2019-09-04 17:00:00,2019-09-04 18:00:00,2019-09-04 19:00:00,2019-09-04 20:00:00,2019-09-04 21:00:00,2019-09-04 22:00:00,2019-09-04 23:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,108.237828,58.244566,52.456770,32.791156,25.772362,27.289895,22.204257,19.933832,20.604757,...,4.800711e+01,4.800711e+01,4.800711e+01,4.800711e+01,4.800711e+01,4.801512e+01,4.801512e+01,4.801512e+01,4.801512e+01,4.801512e+01
1,0x1c9d08cd16fce04790ef900695861e786,7.415428,5.404350,5.184286,3.914106,3.162559,3.090970,2.402750,0.851465,0.928284,...,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11,1.922762e+11
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,17.977447,7.586570,7.248970,5.526705,6.345112,6.744465,6.008902,5.432672,4.942715,...,1.219413e+01,1.219413e+01,1.219413e+01,1.219413e+01,1.219413e+01,1.207853e+01,1.207853e+01,1.207853e+01,1.207853e+01,1.207853e+01
3,0x20158d36236a640cf0524dba149459169,129.206525,56.670246,43.788488,40.362859,36.318655,33.327053,28.593457,23.798991,21.047506,...,6.408693e+01,6.408693e+01,6.408693e+01,6.408693e+01,6.408693e+01,6.408741e+01,6.408741e+01,6.408741e+01,6.408741e+01,6.408741e+01
4,0xc305005dcb1ed6128d816954c5ab9e7e,71.390572,34.138158,36.786351,21.129550,14.582090,28.826650,35.877575,23.798011,29.945079,...,2.771043e+01,2.771043e+01,2.771043e+01,2.771043e+01,2.771043e+01,2.752175e+01,2.752175e+01,2.752175e+01,2.752175e+01,2.752175e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,1.969294,1.615716,1.546162,1.224698,1.006334,1.018727,0.987119,0.619406,0.761786,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
57,0x16d1816bc5d185c47de080d4c6a64bc9a,4.685548,3.248489,3.786442,4.280627,3.683102,3.321422,3.860559,4.267627,4.196557,...,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11,2.929460e+11
58,0x14f480f24c435af1b8574c1c6bab38a1c,8.552452,4.899460,4.429721,4.068588,3.825502,3.389524,3.786012,2.861039,2.372793,...,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12,1.403592e+12
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,1.971574,1.390013,1.234204,0.863781,0.786682,0.738695,0.795932,0.838963,0.715173,...,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11,1.477253e+11


In [232]:
daily = pd.read_csv("/home/max/PycharmProjects/ai-cup-makcfd/Linear_regression/sample_submission_DAILY_max_new_features_FILTERED_float32_ROUND_CLIPPED.csv")
hourly = pd.read_csv("/home/max/PycharmProjects/ai-cup-makcfd/Linear_regression/sample_submission_HOURLY_max_eval_new_features_FILTERED_float32_ROUND.csv")

In [253]:
daily = pd.read_csv("/home/max/PycharmProjects/ai-cup-makcfd/Linear_regression/sample_submission_DAILY_max_new_features_FILTERED_float32_ROUND_CLIPPED.csv")

In [267]:
create_submission(df_from_list2, hourly2)

wrote submission-2022-07-09_21-56-40.120700.zip
